<a href="https://colab.research.google.com/github/Saurav-Somani/Soft_Computing/blob/main/SC_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
from sklearn.model_selection import train_test_split

In [2]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Loading dataset
df_Iris=pd.read_csv("/content/drive/My Drive/SC/iris.csv", names=['A1','A2','A3','A4','A5'])

In [6]:
#Dimensions of Dataset
df_Iris.shape

(150, 5)

In [7]:
#Printing 1st five tuples
df_Iris.head()

,A1,A2,A3,A4,A5
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [8]:
#Normalizing Labels
labelencoder = LabelEncoder()                         
df_Iris['A5'] = labelencoder.fit_transform(df_Iris['A5'])

In [9]:
#normalizing the dataset
min_max = preprocessing.MinMaxScaler()                       
scaled_df_Iris = min_max.fit_transform(df_Iris.values)
final_df_Iris = pd.DataFrame(scaled_df_Iris,columns=["A1","A2","A3","A4","A5"])

In [10]:
#Printing 1st five rows after Normalizing the Dataset
final_df_Iris.head()

,A1,A2,A3,A4,A5
0,0.222222,0.625000,0.067797,0.041667,0.0
1,0.166667,0.416667,0.067797,0.041667,0.0
2,0.111111,0.500000,0.050847,0.041667,0.0
3,0.083333,0.458333,0.084746,0.041667,0.0
4,0.194444,0.666667,0.067797,0.041667,0.0


In [11]:
#Labels will be same as the original dataset
final_df_Iris['A5']=df_Iris['A5']       

In [12]:
final_df_Iris.head()

,A1,A2,A3,A4,A5
0,0.222222,0.625000,0.067797,0.041667,0
1,0.166667,0.416667,0.067797,0.041667,0
2,0.111111,0.500000,0.050847,0.041667,0
3,0.083333,0.458333,0.084746,0.041667,0
4,0.194444,0.666667,0.067797,0.041667,0


In [13]:
#Converting the final dataset to .csv file
final_df_Iris.to_csv('iris2.csv',index=False, header=False)

In [14]:
#Loading the pre-processed dataset
data_iris=df_Iris=pd.read_csv('iris2.csv', header=None)

In [15]:
#Loading .csv file
def load_csv(filename):
    df = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for tuple in csv_reader:
            if not tuple:
                continue
            df.append(tuple)
    return df

In [16]:
#Function for converting string values to float
def str_col_to_float(dataset, col):
    for tuple in dataset:
        tuple[col] = float(tuple[col].strip())

In [17]:
#Function for converting string values to int
def str_col_to_int(dataset, col):
    class_val = [tuple[col] for tuple in dataset]
    unique_val = set(class_val)
    lookup_val = dict()
    for i, value in enumerate(unique_val):
        lookup_val[value] = i
    for tuple in dataset:
        tuple[col] = lookup_val[tuple[col]]
    return lookup_val

In [18]:
#Creating Accuracy Matrix 
def accuracy_metric(act_value, pred_value):
    count = 0
    for i in range(len(act_value)):
        if act_value[i] == pred_value[i]:
            count += 1
    return count / float(len(act_value)) * 100.0

In [19]:
def algo_evaluation(training_dataset, test_dataset, *args):
    test_data = list()   
    act_value = [tuple[-1] for tuple in test_dataset]
    for tuple in test_dataset:
        tuple_c = tuple
        tuple_c[-1] = None
        test_data.append(tuple_c)
    pred_value = back_propagation(training_dataset, test_data, *args)
    accuracy = accuracy_metric(act_value, pred_value)
    return accuracy

In [20]:
#Calculation Activate of a Neuron
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [21]:
#using the sigmoid function to transfer the activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [22]:
#Forward Propagation
def forward_propagation(network, tuple):
    inputs = tuple
    for layer in network:
        next_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            next_inputs.append(neuron['output'])
        inputs = next_inputs
    return inputs

In [23]:
#Calculating derivative of an output
def derivative_transfer(output):
    return output * (1.0 - output)

In [24]:
#Calculation Backward Propagation Error
def backward_propagation_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * derivative_transfer(neuron['output'])

In [25]:
#Updating Weights
def weights_updation(network, tuple, learning_rate):
    for i in range(len(network)):
        inputs = tuple[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += learning_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += learning_rate * neuron['delta']

In [26]:
#Training the Network
def train_network(network, train, learning_rate, num_epoch, n_outputs):
    for epoch in range(num_epoch):
        for tuple in train:
            outputs = forward_propagation(network, tuple)
            expected = [0 for i in range(n_outputs)]
            expected[tuple[-1]] = 1
            backward_propagation_error(network, expected)
            weights_updation(network, tuple, learning_rate)

In [27]:
#Initializing the Network
def network_init(n_inputs, num_hidden, n_outputs):
    net = list()
    hidden_layer = [{'weights':[random() for j in range(n_inputs + 1)]} for j in range(num_hidden)]
    net.append(hidden_layer)
    output_layer = [{'weights':[random() for j in range(num_hidden + 1)]} for j in range(n_outputs)]
    net.append(output_layer)
    return net

In [28]:
#returns the index in the network output that has the largest probability
def predict(network, tuple):
    outputs = forward_propagation(network, tuple)
    return outputs.index(max(outputs))

In [29]:
#Back-Propagation
def back_propagation(train, test, learning_rate, num_epoch, num_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([tuple[-1] for tuple in train]))
    network = network_init(n_inputs, num_hidden, n_outputs)
    train_network(network, train, learning_rate, num_epoch, n_outputs)
    predictions = list()
    for tuple in test:
        prediction = predict(network, tuple)
        predictions.append(prediction)
    return(predictions)

In [30]:
seed(1)
scores=list()

for j in range(20):
    train, test = train_test_split(data_iris, test_size=0.3, shuffle=True)
    train.to_csv('iris_train.csv', index=False, header=False)
    test.to_csv('iris_test.csv', index=False, header=False)
    filename = 'iris_train.csv'
    training_dataset = load_csv(filename)
    filename='iris_test.csv'
    test_dataset = load_csv(filename)
    
    for i in range(len(training_dataset[0])-1):
        str_col_to_float(training_dataset, i)
    str_col_to_int(training_dataset, len(training_dataset[0])-1)
    
    for i in range(len(test_dataset[0])-1):
        str_col_to_float(test_dataset, i)
    str_col_to_int(test_dataset, len(test_dataset[0])-1)

    learning_rate = 0.5
    num_epoch = 1500
    num_hidden = 7
    accuracy = algo_evaluation(training_dataset, test_dataset, learning_rate, num_epoch, num_hidden)
    scores.append(accuracy)
    print('Iteration %s' % (j+1))
    print('Accuracy: %.3f%%'% accuracy)
print (scores)
print('The Mean Accuracy is: %.3f%%' % (sum(scores)/20))

Iteration 1
Accuracy: 97.778%
Iteration 2
Accuracy: 100.000%
Iteration 3
Accuracy: 88.889%
Iteration 4
Accuracy: 93.333%
Iteration 5
Accuracy: 93.333%
Iteration 6
Accuracy: 91.111%
Iteration 7
Accuracy: 91.111%
Iteration 8
Accuracy: 91.111%
Iteration 9
Accuracy: 93.333%
Iteration 10
Accuracy: 93.333%
Iteration 11
Accuracy: 93.333%
Iteration 12
Accuracy: 97.778%
Iteration 13
Accuracy: 95.556%
Iteration 14
Accuracy: 93.333%
Iteration 15
Accuracy: 86.667%
Iteration 16
Accuracy: 100.000%
Iteration 17
Accuracy: 95.556%
Iteration 18
Accuracy: 93.333%
Iteration 19
Accuracy: 95.556%
Iteration 20
Accuracy: 97.778%
[97.77777777777777, 100.0, 88.88888888888889, 93.33333333333333, 93.33333333333333, 91.11111111111111, 91.11111111111111, 91.11111111111111, 93.33333333333333, 93.33333333333333, 93.33333333333333, 97.77777777777777, 95.55555555555556, 93.33333333333333, 86.66666666666667, 100.0, 95.55555555555556, 93.33333333333333, 95.55555555555556, 97.77777777777777]
The Mean Accuracy is: 94.111%
